## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [3]:
import numpy as np
import os
from random import shuffle
import re

In [4]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [5]:
import urllib.request
import zipfile
import lxml.etree

In [6]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [7]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [8]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [9]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [10]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [11]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [12]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [13]:
len(sentences_ted)

266694

In [14]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [15]:
counts_ted={}
for sentence in sentences_ted:
    for word in sentence:
        if word not in counts_ted:
            counts_ted[word]=1
        if word in counts_ted:
            counts_ted[word]+=1

words_ted_top1000=sorted(counts_ted, key=lambda x:int(counts_ted[x]), reverse=True)[:1000]
counts_ted_top1000=[]
for word in words_ted_top1000:
    counts_ted_top1000.append(counts_ted[word])
print(counts_ted_top1000[:10])

    

[207749, 149306, 125170, 114819, 105400, 95147, 83181, 78071, 74739, 70924]


Plot distribution of top-1000 words

In [16]:
hist, edges = np.histogram(np.log(counts_ted_top1000), density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [17]:
from gensim.models import Word2Vec

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [18]:
model_ted = Word2Vec(sentences_ted, size=100, window=5, min_count=5, workers=4)

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [19]:
model_ted.most_similar("man")

[('woman', 0.8434553146362305),
 ('guy', 0.8334395885467529),
 ('boy', 0.7573233842849731),
 ('lady', 0.7547436356544495),
 ('soldier', 0.7478539943695068),
 ('girl', 0.7353519797325134),
 ('gentleman', 0.7251615524291992),
 ('poet', 0.6711980104446411),
 ('john', 0.6692219376564026),
 ('kid', 0.6592816114425659)]

In [20]:
model_ted.most_similar("computer")

[('machine', 0.7319157719612122),
 ('software', 0.712874174118042),
 ('program', 0.6924588680267334),
 ('3d', 0.6876763701438904),
 ('robot', 0.667427659034729),
 ('printer', 0.6626852750778198),
 ('video', 0.6560510396957397),
 ('device', 0.6526011228561401),
 ('chip', 0.6485830545425415),
 ('camera', 0.6381588578224182)]

In [21]:
model_ted.most_similar("animal")

[('organism', 0.7644995450973511),
 ('organ', 0.735634446144104),
 ('ant', 0.6900226473808289),
 ('neuron', 0.6851450204849243),
 ('gene', 0.6747211217880249),
 ('object', 0.6711485385894775),
 ('protein', 0.646078884601593),
 ('virus', 0.6381279230117798),
 ('ecosystem', 0.6279876828193665),
 ('individual', 0.6259214878082275)]

#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [22]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_ted = words_ted_top1000
words_top_vec_ted = model_ted[words_top_ted]

In [23]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [24]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [25]:
if not os.path.isfile('wikitext.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [26]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [27]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
del input_text

In [28]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))

4267112
853422


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [47]:
counts_wiki = {}
for sentence in sentences_wiki:
    for word in sentence:
        if word in counts_wiki:
            counts_wiki[word] += 1
        else:
            counts_wiki[word] = 1

In [53]:
words_top_wiki = sorted(counts_wiki, reverse=True, key=counts_wiki.__getitem__)
counts_wiki_top1000 = [counts_wiki[word] for word in words_top_wiki[:1000]]
print(counts_wiki_top1000[:10])

[24831720, 9901755, 7150853, 7029977, 5980974, 5959059, 5843069, 5393472, 5347208, 3969734]


In [56]:
hist, edges = np.histogram(counts_wiki_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

In [57]:
model_wiki = Word2Vec(sentences_ted, size=100, window=5, min_count=5, workers=4)

In [58]:
model_wiki.most_similar("man")

[('woman', 0.8464617729187012),
 ('guy', 0.8078672885894775),
 ('lady', 0.7836194038391113),
 ('boy', 0.7650604844093323),
 ('girl', 0.7449390292167664),
 ('soldier', 0.7323599457740784),
 ('gentleman', 0.7263708114624023),
 ('kid', 0.6921200752258301),
 ('friend', 0.6827057600021362),
 ('surgeon', 0.6762031316757202)]

In [59]:
model_wiki.most_similar("computer")

[('machine', 0.7470157146453857),
 ('software', 0.7153013348579407),
 ('robot', 0.692807674407959),
 ('device', 0.6856371164321899),
 ('camera', 0.6658025979995728),
 ('program', 0.6644234657287598),
 ('video', 0.6214884519577026),
 ('3d', 0.6191324591636658),
 ('game', 0.6146800518035889),
 ('satellite', 0.6107714176177979)]

In [60]:
model_wiki.most_similar("animal")

[('organism', 0.7531567811965942),
 ('organ', 0.7248586416244507),
 ('ant', 0.7223650813102722),
 ('insect', 0.6938428282737732),
 ('object', 0.6704449653625488),
 ('neuron', 0.6651753187179565),
 ('colony', 0.658514142036438),
 ('gene', 0.6557891368865967),
 ('virus', 0.6440666913986206),
 ('algorithm', 0.6403990983963013)]

#### t-SNE visualization

In [65]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

words_top_wiki = counts_wiki_top1000
words_top_vec_wiki = model_wiki[words_top_wiki]

KeyError: "word ' ' not in vocabulary"

In [67]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki[:1000]))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

NameError: name 'words_top_wiki_tsne' is not defined